In [3]:
from bs4 import BeautifulSoup
 
from googletrans import Translator

import pandas as pd

In [31]:
# Reading data from the xml file
with open('CV_03_04.xml', 'r', encoding='utf-8') as f:
    data = f.read()
 

In [7]:
translator = Translator()

In [41]:

bs_data = BeautifulSoup(data, 'xml')
 
b_unique = bs_data.find_all('ATIVIDADES-DE-ENSINO')
mes_inicio = []
ano_inicio = []
mes_fim = []
ano_fim = []
curso = []
nivel = []
instituicao = []
disciplina = []
for i in b_unique:
    r = i.find_all('ENSINO')
    inst = i.find_parent().get('NOME-INSTITUICAO')
    for e in r:
        mes_inicio.append(e.get('MES-INICIO'))
        ano_inicio.append(e.get('ANO-INICIO'))
        mes_fim.append(e.get('MES-FIM'))
        ano_fim.append(e.get('ANO-FIM'))


        nivel.append(e.get('TIPO-ENSINO'))

        instituicao.append(translator.translate(inst, dest='en').text)
        
        curso.append(translator.translate(e.get('NOME-CURSO'), dest='en').text)

        

        disciplinas=''
        for j in e:
            
            d = j.get_text()
            if(disciplinas==''):
                disciplinas =translator.translate(d, dest='en').text.capitalize()
            else:
                if(d!=''):
                    disciplinas =disciplinas+'; '+translator.translate(d.replace(';', ''), dest='en').text.capitalize()
            
        disciplina.append(disciplinas)
    
df = pd.DataFrame()
df['mes_inicio']= mes_inicio
df['ano_inicio']= ano_inicio
df['mes_fim']= mes_fim
df['ano_fim']= ano_fim
df['curso']= curso 
df['nivel']= nivel
df['instituicao']= instituicao
df['disciplina']= disciplina


In [43]:
df.head()

,mes_inicio,ano_inicio,mes_fim,ano_fim,curso,nivel,instituicao,disciplina
0,07,2010,12,2010,Information Technology Management,GRADUACAO,Paulista University,"Costs, benefits and organizational impacts of ..."
1,01,2011,04,2011,Information Technology Management,GRADUACAO,Paulista University,"Costs, benefits and organizational impacts of ..."
2,01,2014,06,2014,computer Science,GRADUACAO,University of the Amazon,Programming logic; Computational intelligence;...
3,01,2014,06,2014,Development of Software Based Systems. live,ESPECIALIZACAO,University of the Amazon,Knowledge engineering
4,06,2013,12,2013,computer Science,GRADUACAO,University of the Amazon,Formal languages ​​and automata; Computational...


In [44]:

df['curso'] = df['curso'].apply(lambda x: x.title())

In [45]:
def sub_current (x):
    s = x
    if(x==''):
        s= 'Current'
    return s
df['ano_fim'] = df['ano_fim'].apply(lambda x: sub_current(x))
set(df.ano_fim.values)

{'2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 'Current'}

In [49]:
df = df.sort_values(by=['ano_fim', 'mes_fim'], ascending=False)

In [51]:
df['disciplina'] = df['disciplina'].apply(lambda x: x.replace(';;', ';'))
    
for index, row in df.iterrows():
    print(row.disciplina)

Formal languages ​​and automata
Formal languages ​​and automata
Scientific methodology; Special topics in artificial intelligence
Scientific methodology
Artificial intelligence
Data science
Formal languages ​​and automata
Formal languages ​​and automata
Machine learning
Scientific methodology; Advanced topics in artificial intelligence
Advanced topics in artificial intelligence
Integrative seminar
Formal languages ​​and automata
Formal languages ​​and automata
Special topics in computational intelligence
Special topics in machine learning
Technologies
Data structures i; Formal languages ​​and automata
Data structures i; Web programming
Scientific methodology
Special topics in computational intelligence
Technologies
Formal languages ​​and automata
Formal languages ​​and automata
Special topics in computational intelligence
Scientific methodology; Formal languages ​​and automata
Data structure i; Formal languages ​​and automata
Introduction to computer science; Computing theory
Data stru

In [52]:
df.columns

Index(['mes_inicio', 'ano_inicio', 'mes_fim', 'ano_fim', 'curso', 'nivel',
       'instituicao', 'disciplina'],
      dtype='object')

In [53]:
df.head(2)

,mes_inicio,ano_inicio,mes_fim,ano_fim,curso,nivel,instituicao,disciplina
30,02,2023,,Current,Information Systems,GRADUACAO,Federal University of Western Pará,Formal languages ​​and automata
31,02,2023,,Current,Bachelor'S Degree In Computer Science,GRADUACAO,Federal University of Western Pará,Formal languages ​​and automata


In [65]:
set(df.nivel.values)

{'ESPECIALIZACAO', 'GRADUACAO', 'POS-GRADUACAO'}

In [69]:
for index, row in df.iterrows():
    if(str(row.ano_fim) == 'Current'):
       s = '\\item['+row.mes_inicio+'/' + row.ano_inicio + '--'+ row.ano_fim+']'
    else:
       s = '\\item['+row.mes_inicio+'/' + row.ano_inicio + '--'+ row.mes_fim+'/'+row.ano_fim +']'
    s = s+ "{\\textbf{"+row.instituicao+"}}\\\\"
    s = s+ "\n"+"{Role:} Lecturer \\\\"
    
    if(row.nivel == 'GRADUACAO'):
        grad ="\n{Level: } Undergrad.\\\\"

    if(row.nivel == 'POS-GRADUACAO'):
        grad ="\n{Level: } Grad. (Masters')\\\\"
            
    if(row.nivel == 'ESPECIALIZACAO'):
        grad ="\n{Level: } Grad. (MBA)\\\\"
    s = s+grad
    s = s+ '\n'+"{Program: } "+row.curso+ "\\\\"
    s = s+'\n'+"{Courses:}\n\\begin{itemize}"
    
    for i in row.disciplina.split('; '):
        s = s+ '\n'+"\\item[--]  "+  i
    s = s+ '\n'+"\\end{itemize}"
    
    
    
    print(s)
    


\item[02/2023--Current]{\textbf{Federal University of Western Pará}}\\
{Role:} Lecturer \\
{Level: } Undergrad.\\
{Program: } Information Systems\\
{Courses:}
\begin{itemize}
\item[--]  Formal languages ​​and automata
\end{itemize}
\item[02/2023--Current]{\textbf{Federal University of Western Pará}}\\
{Role:} Lecturer \\
{Level: } Undergrad.\\
{Program: } Bachelor'S Degree In Computer Science\\
{Courses:}
\begin{itemize}
\item[--]  Formal languages ​​and automata
\end{itemize}
\item[09/2022--02/2023]{\textbf{Federal University of Western Pará}}\\
{Role:} Lecturer \\
{Level: } Undergrad.\\
{Program: } Bachelor'S Degree In Computer Science\\
{Courses:}
\begin{itemize}
\item[--]  Scientific methodology
\item[--]  Special topics in artificial intelligence
\end{itemize}
\item[09/2021--02/2022]{\textbf{Federal University of Western Pará}}\\
{Role:} Lecturer \\
{Level: } Undergrad.\\
{Program: } Bachelor'S Degree In Computer Science\\
{Courses:}
\begin{itemize}
\item[--]  Scientific methodolo

In [ ]:


\item[01/2011--04/2011]{\textbf{Paulista University}}\\
{Role:} Lecturer \\
{Level: } Undergrad.\\
{Program: } Information Technology Management\\
{Courses:}
\begin{itemize}
\item[--]  Costs, benefits and organizational impacts of information systems
\item[--]  Database technology for information systems
\item[--]  Software development for information systems
\end{itemize}